In [2]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import nltk
import math

In [3]:
def conv_to_char(n):
    s = list(n)
    st = ""
    for i in s:
        st += chr(i)
    return st

In [4]:
def features_of_images_train_val(location_folder):
    
    list_of_files = os.listdir(location_folder)
    
    full_file_path = []
    for i in list_of_files:
        full_file_path.append(location_folder+i)

    id = []
    cls = []
    image = []

    for i in full_file_path:
        raw_img = tf.data.TFRecordDataset(i)
        image_description = {
            "id" : tf.io.FixedLenFeature([],tf.string),
            "class": tf.io.FixedLenFeature([],tf.int64),
            "image": tf.io.FixedLenFeature([],tf.string)
        }
        y = lambda x: tf.io.parse_single_example(x,image_description)
        img = raw_img.map(y)

        begin = i.rindex("-")+1
        end = i.index(".tfrec")
        num = int(i[begin:end])

        for dr in img.take(num):
            id.append(conv_to_char(dr["id"].numpy()))
            cls.append(int(dr["class"].numpy()))
            image.append(tf.io.decode_image(dr["image"]).numpy())
    
    id = np.array(id)
    cls = np.array(cls)
    image = np.array(image)
    
    return id,cls,image

In [5]:
def features_of_images_test(location_folder):

    list_of_files = os.listdir(location_folder)

    full_file_path = []
    for i in list_of_files:
        full_file_path.append(location_folder+i)

    id = []
    image = []

    for i in full_file_path:
        raw_img = tf.data.TFRecordDataset(i)
        image_description = {
            "id" : tf.io.FixedLenFeature([],tf.string),
            "image": tf.io.FixedLenFeature([],tf.string)
        }
        y = lambda x: tf.io.parse_single_example(x,image_description)
        img = raw_img.map(y)

        begin = i.rindex("-")+1
        end = i.index(".tfrec")
        num = int(i[begin:end])

        for dr in img.take(num):
            id.append(conv_to_char(dr["id"].numpy()))
            image.append(tf.io.decode_image(dr["image"]).numpy())

    id = np.array(id)
    image = np.array(image)
    
    return id,image

In [6]:
location_folder_train_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/train/"

location_folder_train_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/train/"

location_folder_train_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/train/"

location_folder_train_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/train/"

In [7]:
location_folder_val_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/val/"

location_folder_val_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/val/"

location_folder_val_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/val/"

location_folder_val_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/val/"

In [8]:
location_folder_test_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/test/"

location_folder_test_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/test/"

location_folder_test_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/test/"

location_folder_test_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/test/"

### Training Dataset

In [9]:
id_train_192,cls_train_192,image_train_192 = features_of_images_train_val(location_folder_train_192)

In [10]:
id_train_224,cls_train_224,image_train_224 = features_of_images_train_val(location_folder_train_224)

In [11]:
id_train_331,cls_train_331,image_train_331 = features_of_images_train_val(location_folder_train_331)

In [12]:
id_train_512,cls_train_512,image_train_512 = features_of_images_train_val(location_folder_train_512)

### Validation Dataset

In [13]:
id_val_192,cls_val_192,image_val_192 = features_of_images_train_val(location_folder_val_192)

In [14]:
id_val_224,cls_val_224,image_val_224 = features_of_images_train_val(location_folder_val_224)

In [15]:
id_val_331,cls_val_331,image_val_331 = features_of_images_train_val(location_folder_val_331)

In [16]:
id_val_512,cls_val_512,image_val_512 = features_of_images_train_val(location_folder_val_512)

### Testing Dataset

In [17]:
id_test_192,image_test_192 = features_of_images_test(location_folder_test_192)

In [18]:
id_test_224,image_test_224 = features_of_images_test(location_folder_test_224)

In [19]:
id_test_331,image_test_331 = features_of_images_test(location_folder_test_331)

In [20]:
id_test_512,image_test_512 = features_of_images_test(location_folder_test_512)

### Preprocessing

In [21]:
id_192 = np.concatenate([id_train_192,id_val_192,id_test_192])

In [22]:
id_224 = np.concatenate([id_train_224,id_val_224,id_test_224])

In [23]:
id_331 = np.concatenate([id_train_331,id_val_331,id_test_331])

In [24]:
id_512 = np.concatenate([id_train_512,id_val_512,id_test_512])

In [25]:
image_192 = np.concatenate([image_train_192,image_val_192,image_test_192])

In [26]:
image_224 = np.concatenate([image_train_224,image_val_224,image_test_224])

In [ ]:
image_331 = np.concatenate([image_train_331,image_val_331,image_test_331])

In [1]:
image_512 = np.concatenate([image_train_512,image_val_512,image_test_512])

NameError: name 'np' is not defined

In [ ]:
cls_192 = np.concatenate([cls_train_192,cls_val_192])

In [ ]:
cls_224 = np.concatenate([cls_train_224,cls_val_224])

In [ ]:
cls_331 = np.concatenate([cls_train_331,cls_val_331])

In [ ]:
cls_512 = np.concatenate([cls_train_512,cls_val_512])